### Import  libraries

In [24]:
from bs4 import BeautifulSoup
from datetime import datetime
from googletrans import Translator

import requests
import pandas as pd

ModuleNotFoundError: No module named 'googletrans'

In [4]:
#<table id="holidays-table" class="table table--left table--inner-borders-rows table--full-width table--sticky table--holidaycountry">

### Web Scraping

In [5]:

# URL of the webpage to be scraped
url = 'https://www.timeanddate.com/holidays/spain/2013'


response = requests.get(url)

if response.status_code == 200:
   
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract specific data
  
    th_tags = soup.find_all('th')

    for tag in th_tags:
        print(tag.text)

    links = soup.find_all('a')
    for link in links:
        print(link.get('href'))
else:
    print(f'Failed to retrieve the webpage. Status code: {response.status_code}')


Date
 
Name
Type
Details
1 जनवरी
2 जनवरी
6 जनवरी
7 जनवरी
11 फरवरी
12 फरवरी
13 फरवरी
14 फरवरी
28 फरवरी
1 मार्च
13 मार्च
19 मार्च
19 मार्च
19 मार्च
20 मार्च
24 मार्च
28 मार्च
28 मार्च
29 मार्च
30 मार्च
31 मार्च
31 मार्च
1 अप्रैल
1 अप्रैल
23 अप्रैल
23 अप्रैल
23 अप्रैल
1 मई
2 मई
5 मई
17 मई
19 मई
20 मई
30 मई
30 मई
31 मई
9 जून
9 जून
10 जून
10 जून
13 जून
21 जून
24 जून
24 जून
25 जुलाई
25 जुलाई
28 जुलाई
5 अगस्त
8 अगस्त
11 अगस्त
15 अगस्त
2 सितंबर
6 सितंबर
8 सितंबर
8 सितंबर
8 सितंबर
9 सितंबर
11 सितंबर
15 सितंबर
17 सितंबर
22 सितंबर
9 अक्टूबर
12 अक्टूबर
15 अक्टूबर
27 अक्टूबर
1 नवंबर
3 दिसंबर
6 दिसंबर
8 दिसंबर
9 दिसंबर
21 दिसंबर
24 दिसंबर
25 दिसंबर
29 दिसंबर
31 दिसंबर
/
/
/news/
/news
/news/astronomy/
/news/time/
/news/calendar/
/newsletter/
/live/
/worldclock/
/worldclock/
/worldclock/full.html
/worldclock/personal.html
/clocks/free.html
/worldclock/timezone/utc
/time/
/time
/worldclock/converter.html
/worldclock/meeting.html
/worldclock/fixedform.html
/time/map/
/time/zones/
/time/dst/
/time/chang

In [18]:
th_tags=th_tags[:3]
print(th_tags)

[<th rowspan="2">Date</th>, <th rowspan="2"> </th>, <th rowspan="2">Name</th>]


In [19]:
word_table_titles = [title.text.strip() for title in th_tags]

print(word_table_titles)

['Date', '', 'Name']


In [20]:
df = pd.DataFrame(columns = word_table_titles)
print(df)

Empty DataFrame
Columns: [Date, , Name]
Index: []


In [21]:
column_data = soup.find_all('tr')

In [22]:
column_data

[<tr><th rowspan="2">Date</th><th rowspan="2"> </th><th rowspan="2">Name</th><th rowspan="2">Type</th><th rowspan="2">Details</th></tr>,
 <tr></tr>,
 <tr id="hol_jan"></tr>,
 <tr data-date="1356998400000" data-mask="1" id="tr1"><th class="nw">1 जनवरी</th><td class="nw">मंगलवार</td><td><a href="/holidays/spain/new-year-day">New Year's Day</a></td><td>National holiday</td><td> </td></tr>,
 <tr data-date="1357084800000" data-mask="32" id="tr2"><th class="nw">2 जनवरी</th><td class="nw">बुधवार</td><td><a href="/holidays/spain/reconquest-day">Reconquest Day</a></td><td>Observance</td><td> </td></tr>,
 <tr data-date="1357430400000" data-mask="1" id="tr3"><th class="nw">6 जनवरी</th><td class="nw">रविवार</td><td><a href="/holidays/spain/epiphany">Epiphany</a></td><td>National holiday</td><td> </td></tr>,
 <tr data-date="1357516800000" data-mask="4194304" id="tr4"><th class="nw">7 जनवरी</th><td class="nw">सोमवार</td><td><a href="/holidays/spain/epiphany-observed">Epiphany observed</a></td><td>Co

In [38]:
rows = soup.find_all('tr')

df = pd.DataFrame(columns=['Date', 'Name'])

# Function to convert Hindi month name to English and format date as dd-mm-yyyy
def convert_hindi_to_english(date_str):
    months_hindi_to_english = {
        "जनवरी": "01",
        "फरवरी": "02",
        "मार्च": "03",
        "अप्रैल": "04",
        "मई": "05",
        "जून": "06",
        "जुलाई": "07",
        "अगस्त": "08",
        "सितंबर": "09",
        "अक्टूबर": "10",
        "नवंबर": "11",
        "दिसंबर": "12"
    }
    
    # Split date_str into day and month
    parts = date_str.split()
    day = parts[0]
    month_hindi = parts[1]
    
    # Convert month to English and format day as two digits
    month = months_hindi_to_english.get(month_hindi, '01') 
    formatted_date = f"{day.zfill(2)}-{month}-2013"
    
    return formatted_date

data = []

for row in rows[1:]: 
    cols = row.find_all(['th', 'td'])
    if len(cols) == 5: 
        date_hindi = cols[0].text.strip()
        name = cols[2].text.strip()
        
        # Convert Hindi date to formatted English date
        date_english = convert_hindi_to_english(date_hindi)
        
        data.append([date_english, name])


df = pd.DataFrame(data, columns=['Date', 'Name'])


print(df)

          Date                      Name
0   01-01-2013            New Year's Day
1   02-01-2013            Reconquest Day
2   06-01-2013                  Epiphany
3   07-01-2013         Epiphany observed
4   11-02-2013           Carnival Monday
..         ...                       ...
70  21-12-2013         December Solstice
71  24-12-2013             Christmas Eve
72  25-12-2013             Christmas Day
73  29-12-2013  Feast of the Holy Family
74  31-12-2013            New Year's Eve

[75 rows x 2 columns]
